In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
def progress_bar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd="\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()

## LSTM

In [195]:
# Feature engineering: Add time-related features and merge vessel data
def preprocess_data(ais_train, vessels, seq_len=30):

    # remove all vessels with less than seq_len observations
    ais_train = ais_train.groupby('vesselId').filter(lambda x: len(x) > seq_len)

    # Sort by vesselId and time to ensure correct order before calculating time delta
    ais_train = ais_train.sort_values(by=['vesselId', 'time'])

    # Convert 'time' to datetime format
    ais_train['time'] = pd.to_datetime(ais_train['time'])

    # Calculate time_delta in second. This is the next time minus the current time
    ais_train['time_delta'] = ais_train.groupby('vesselId')['time'].diff().dt.total_seconds()
    # shift the time_delta by one to get the time_delta of the next observation
    ais_train['time_delta'] = ais_train.groupby('vesselId')['time_delta'].shift(-1)

    # Extract other time-based features
    newyear = pd.to_datetime('2024-01-01 00:00:00')
    ais_train['time_numeric'] = (ais_train['time'] - newyear).dt.total_seconds()
    ais_train['month'] = ais_train['time'].dt.month
    ais_train['hour'] = ais_train['time'].dt.hour
    ais_train['weekday'] = ais_train['time'].dt.weekday

    # make a dictionary with vesselId as key and a unique number as value
    vesselId = ais_train['vesselId'].unique()
    vesselId_dict = {vesselId[i]: i for i in range(len(vesselId))}
    ais_train['vessel_embedding'] = ais_train['vesselId'].map(vesselId_dict)

    # Merge vessel data (on vesselId) to add CEU, length, vesselType
    ais_train = ais_train.merge(vessels[['vesselId', 'CEU', 'length', 'vesselType']], on='vesselId', how='left')

    # all the missing values for vesselType are made into a new category
    ais_train['vesselType'] = ais_train['vesselType'].fillna(-1)

    # Handle missing values after merging, if necessary
    ais_train.fillna(0, inplace=True)

    # Drop columns that are not needed
    ais_train = ais_train.drop(columns=['time', 'rot', 'heading', 'etaRaw', 'vesselId', 'portId'])

    return ais_train, vesselId_dict

In [196]:
X_train = pd.read_csv('../data/ais_train_clustered.csv', sep=',')
vessels = pd.read_csv('../data/vessels.csv', sep='|')

X_train, vesselId_dict = preprocess_data(X_train, vessels)
X_train.tail()

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,weekday,vessel_embedding,CEU,length,vesselType
1522059,324.1,13.5,0,59.63337,21.43237,11054176.0,7.0,1249.0,5,22,1,686,200,191.0,83.0
1522060,324.2,13.3,0,59.69588,21.34225,11055425.0,7.0,1249.0,5,22,1,686,200,191.0,83.0
1522061,356.5,12.2,0,59.76388,21.35317,11056674.0,7.0,1219.0,5,23,1,686,200,191.0,83.0
1522062,52.6,17.3,0,59.83316,21.38489,11057893.0,7.0,1248.0,5,23,1,686,200,191.0,83.0
1522063,53.6,17.7,0,59.89167,21.54685,11059141.0,7.0,0.0,5,23,1,686,200,191.0,83.0


In [191]:
# print where vessel_embedding is 65
X_train[X_train['vessel_embedding'] == 322].tail()

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,weekday,vessel_embedding,CEU,length,vesselType
584572,338.3,17.8,0,-33.76641,17.73318,10748270.0,5.0,1212.0,5,9,5,322,6312,199.99,83.0
584573,338.5,17.9,0,-33.67195,17.69076,10749482.0,5.0,1242.0,5,9,5,322,6312,199.99,83.0
584574,340.8,18.0,0,-33.57465,17.64701,10750724.0,5.0,1098.0,5,10,5,322,6312,199.99,83.0
584575,339.4,18.4,0,-33.48759,17.60807,10751822.0,5.0,785.0,5,10,5,322,6312,199.99,83.0
584576,337.1,18.3,0,-33.42449,17.57977,10752607.0,5.0,0.0,5,10,5,322,6312,199.99,83.0


In [197]:
# Store the vesselId_dict with pickle
with open('../data/xgb_kalman/vesselId_dict.pkl', 'wb') as f:
    pickle.dump(vesselId_dict, f)


## Add kalman

In [19]:
%pip install filterpy

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110542 sha256=5cabeded3e5571a879c0da93ffe8403945eb7da970c5b9e86c8d1e8419992470
  Stored in directory: c:\users\simen\appdata\local\pip\cache\wheels\12\dc\3c\e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.


In [201]:
import numpy as np
import pandas as pd
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import MerweScaledSigmaPoints
from geopy.distance import distance
from geopy import Point

# Function to compute distance from SOG and delta_t
def compute_distance(sog_knots, delta_t_seconds):
    # Convert SOG from knots to meters per second (1 knot = 0.514444 m/s)
    sog_mps = sog_knots * 0.514444
    distance_m = sog_mps * delta_t_seconds
    return distance_m

# State transition function for UKF
def fx(x, dt):
    lat = x[0]
    lon = x[1]
    sog = x[2]
    cog = x[3]

    # Ensure SOG and COG are valid
    if sog <= 0 or sog >= 1022 or np.isnan(sog):
        sog = x[2]  # Use previous valid SOG
    if cog < 0 or cog >= 360 or np.isnan(cog):
        cog = x[3]  # Use previous valid COG

    # Compute distance traveled
    distance_m = compute_distance(sog, dt)

    # Use geopy to compute new position
    start_point = Point(lat, lon)
    destination = distance(meters=distance_m).destination(point=start_point, bearing=cog)

    lat_new = destination.latitude
    lon_new = destination.longitude

    # Assume SOG and COG remain the same
    sog_new = sog
    cog_new = cog

    return np.array([lat_new, lon_new, sog_new, cog_new])

# Measurement function for UKF
def hx(x):
    return x

# Assuming your DataFrame is named X_train and includes 'vessel_id' column
# Create empty list to store results
results = []

# Define default values for SOG and COG
default_sog = 10.0  # Example default speed in knots
default_cog = 0.0   # Example default course in degrees

k = 0
n_iter = X_train['vessel_embedding'].nunique()

kalman_filters_dict = {}

# Group data by 'vessel_id'
for vessel_id, group in X_train.groupby('vessel_embedding'):
    # Sort the group by time
    group = group.sort_values('time_numeric').reset_index(drop=True)
    n_timesteps = len(group)

    # Define the state dimension and measurement dimension
    dim_x = 4  # [latitude, longitude, sog, cog]
    dim_z = 4  # Measurements: same as state

    # Define the sigma points
    points = MerweScaledSigmaPoints(n=dim_x, alpha=0.1, beta=2.0, kappa=0)

    # Initialize the UKF
    kf = UKF(dim_x=dim_x, dim_z=dim_z, fx=fx, hx=hx, dt=1.0, points=points)

    # Initial state: latitude, longitude, SOG, COG
    initial_lat = group.loc[0, 'latitude']
    initial_lon = group.loc[0, 'longitude']
    initial_sog = group.loc[0, 'sog']
    initial_cog = group.loc[0, 'cog']

    # Handle missing initial SOG or COG using bfill()
    if np.isnan(initial_sog) or initial_sog >= 1022 or initial_sog <= 0:
        initial_sog = group['sog'].bfill().iloc[0]
        if np.isnan(initial_sog) or initial_sog >= 1022 or initial_sog <= 0:
            initial_sog = default_sog

    if np.isnan(initial_cog) or initial_cog >= 360 or initial_cog < 0:
        initial_cog = group['cog'].bfill().iloc[0]
        if np.isnan(initial_cog) or initial_cog >= 360 or initial_cog < 0:
            initial_cog = default_cog

    kf.x = np.array([initial_lat, initial_lon, initial_sog, initial_cog])

    # Initial covariance
    kf.P *= 10.

    # Process noise covariance
    kf.Q = np.diag([0.0001, 0.0001, 0.1, 0.1])  # Adjust these values based on your data

    # Measurement noise covariance
    kf.R = np.diag([0.0001, 0.0001, 0.1, 0.1])  # Adjust these values based on your data

    # Initialize the lists to store estimates
    lat_kalman = []
    lon_kalman = []
    sog_kalman = []
    cog_kalman = []

    n_timesteps = len(group)

    for i in range(n_timesteps - 1):
        kf.dt = group.loc[i, 'time_delta']

        # Predict step
        kf.predict()

        # Measurement update
        latitude_meas = group.loc[i+1, 'latitude']
        longitude_meas = group.loc[i+1, 'longitude']
        sog_meas = group.loc[i+1, 'sog']
        cog_meas = group.loc[i+1, 'cog']

        # Validate measurements
        valid_measurement = True
        if (np.isnan(latitude_meas) or np.isnan(longitude_meas) or
            np.isnan(sog_meas) or sog_meas >= 1022 or sog_meas <= 0 or
            np.isnan(cog_meas) or cog_meas >= 360 or cog_meas < 0):
            valid_measurement = False

        if valid_measurement:
            z = np.array([latitude_meas, longitude_meas, sog_meas, cog_meas])
            kf.update(z)
        else:
            # If measurement is invalid, you can skip the update or use the prediction
            pass

        # Store estimates corresponding to time t_{i+1}
        lat_kalman.append(kf.x[0])
        lon_kalman.append(kf.x[1])
        sog_kalman.append(kf.x[2])
        cog_kalman.append(kf.x[3])

    # Adjust the group DataFrame to match the indices
    group = group.iloc[1:].reset_index(drop=True)
    group['latitude_kalman'] = lat_kalman
    group['longitude_kalman'] = lon_kalman
    group['sog_kalman'] = sog_kalman
    group['cog_kalman'] = cog_kalman

    # Append to results
    results.append(group)

    # Store the filter for later use
    kalman_filters_dict[vessel_id] = kf

    # Print progress
    k += 1
    progress_bar(k, n_iter, prefix='Progress:', suffix='Complete', length=50)

# Concatenate the results
X_train_kalman = pd.concat(results)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [202]:
# save the kalman filter
import pickle
with open('../data/xgb_kalman/kalman_filters_dict.pkl', 'wb') as f:
    pickle.dump(kalman_filters_dict, f)

X_train_kalman.to_csv('../data/xgb_kalman/ais_train_kalman.csv', index = False)

In [203]:
X_train_kalman.tail()

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,weekday,vessel_embedding,CEU,length,vesselType,latitude_kalman,longitude_kalman,sog_kalman,cog_kalman
6659,324.1,13.5,0,59.63337,21.43237,11054176.0,7.0,1249.0,5,22,1,686,200,191.0,83.0,59.604671,21.507221,13.988585,312.687220
6660,324.2,13.3,0,59.69588,21.34225,11055425.0,7.0,1249.0,5,22,1,686,200,191.0,83.0,59.661060,21.405233,13.563406,319.802466
6661,356.5,12.2,0,59.76388,21.35317,11056674.0,7.0,1219.0,5,23,1,686,200,191.0,83.0,59.724631,21.373044,12.720960,342.482787
6662,52.6,17.3,0,59.83316,21.38489,11057893.0,7.0,1248.0,5,23,1,686,200,191.0,83.0,59.791703,21.380197,15.551079,163.325389
6663,53.6,17.7,0,59.89167,21.54685,11059141.0,7.0,0.0,5,23,1,686,200,191.0,83.0,59.853465,21.483264,16.879180,95.511281


## Post kalman data preperation

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [4]:
def process_navstat(df):
    """
    Process the 'navstat' feature according to the specified rules:
    - Set all values equal to 8 to 0.
    - Set all values equal to 5 to 1.
    - Set all other values to 15.
    """
    df['navstat'] = df['navstat'].replace(8, 0)
    df['navstat'] = df['navstat'].replace(5, 1)
    df.loc[~df['navstat'].isin([0, 1]), 'navstat'] = 15
    return df


In [5]:
def create_lag_features(df, lagged_vars, n_lag, group_col='vessel_embedding'):
    """
    Create lag features for specified variables.

    Parameters:
    - df: DataFrame containing the data.
    - lagged_vars: List of variable names to create lag features for.
    - n_lag: Number of lag steps to create.
    - group_col: Column name to group by (e.g., 'vessel_id').
    """
    for var in lagged_vars:
        for lag in range(1, n_lag + 1):
            df[f'{var}_lag{lag}'] = df.groupby(group_col)[var].shift(lag)
    return df


In [6]:
def adjust_features_for_prediction_time(df, time_vars, group_col='vessel_embedding'):
    """
    Adjust time-related features to correspond to the prediction time.

    Parameters:
    - df: DataFrame containing the data.
    - time_vars: List of time-related feature names to adjust (e.g., ['month', 'hour', 'weekday']).
    - group_col: Column name to group by (e.g., 'vessel_id').
    """
    for var in time_vars:
        df[var] = df.groupby(group_col)[var].shift(-1)
    return df

In [7]:
def create_target_variables(df, target_vars, group_col='vessel_embedding'):
    """
    Create target variables by shifting the specified variables backward.

    Parameters:
    - df: DataFrame containing the data.
    - target_vars: List of target variable names to create (e.g., ['latitude', 'longitude']).
    - group_col: Column name to group by (e.g., 'vessel_id').
    """
    for var in target_vars:
        df[f'{var}_next'] = df.groupby(group_col)[var].shift(-1)
    return df

In [8]:
def encode_categorical_variables(df, categorical_cols):
    """
    Encode categorical variables using Label Encoding.

    Parameters:
    - df: DataFrame containing the data.
    - categorical_cols: List of categorical column names to encode.

    Returns:
    - df: DataFrame with encoded categorical variables.
    - label_encoders: Dictionary of LabelEncoders used for each column.
    """
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df.loc[:, col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    return df, label_encoders

In [9]:
def prepare_features_and_targets(df, lagged_vars, n_lag):
    """
    Prepare the feature matrix X and target vector y for model training.

    Parameters:
    - df: DataFrame containing the data.
    - lagged_vars: List of variable names for which lag features were created.
    - n_lag: Number of lag steps used.

    Returns:
    - X: Feature matrix.
    - y: Target vector.
    - feature_cols: List of feature column names.
    - target_cols: List of target column names.
    """
    # Feature columns
    lagged_feature_cols = [f'{var}_lag{lag}' for var in lagged_vars for lag in range(1, n_lag + 1)]
    feature_cols = lagged_feature_cols + [
        'month', 'hour', 'weekday', 'vessel_embedding', 'CEU', 'length', 'vesselType'
    ]
    
    # Target columns
    target_cols = ['latitude_next', 'longitude_next', 'sog_next', 'cog_next', 'navstat_next']
    
    # Prepare X and y
    X = df[feature_cols]
    y = df[target_cols]
    
    return X, y, feature_cols, target_cols


In [10]:
def split_data(X, y, df, split_ratio=0.9):
    """
    Split the data into training and testing sets based on time.

    Parameters:
    - X: Feature matrix.
    - y: Target vector.
    - df: Original DataFrame containing 'time_numeric' and 'vessel_id'.
    - split_ratio: Proportion of data to use for training (default is 0.9).

    Returns:
    - X_train, X_test, y_train, y_test: Split data.
    """
    # Sort the DataFrame by 'vessel_id' and 'time_numeric'
    df.sort_values(by=['vessel_embedding', 'time_numeric'], inplace=True)
    
    # Get the indices after sorting
    indices = df.index
    
    # Calculate the split index
    train_size = int(split_ratio * len(df))
    
    # Split indices
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    
    # Split the data
    X_train = X.loc[train_indices]
    X_test = X.loc[test_indices]
    y_train = y.loc[train_indices]
    y_test = y.loc[test_indices]
    
    return X_train, X_test, y_train, y_test


In [23]:
def split_reg_clf_test(X_train, y_train, X_test, y_test, target_vars_reg, target_vars_clf):
    """
    Split the data into regression and classification targets.

    Parameters:
    - X_train: Training feature matrix.
    - y_train: Training target vector.
    - X_test: Testing feature matrix.
    - y_test: Testing target vector.
    - target_vars_reg: List of target variables for regression.
    - target_vars_clf: List of target variables for classification.

    Returns:
    - X_train_reg, y_train_reg, X_test_reg, y_test_reg: Regression data.
    - X_train_clf, y_train_clf, X_test_clf, y_test_clf: Classification data.
    """
    # Add target prefix to target variables
    target_vars_reg = [f'{var}_next' for var in target_vars_reg]
    target_vars_clf = [f'{var}_next' for var in target_vars_clf]

    # Regression data
    X_train_reg = X_train.copy()
    y_train_reg = y_train[target_vars_reg].copy()
    X_test_reg = X_test.copy()
    y_test_reg = y_test[target_vars_reg].copy()
    
    # Classification data
    X_train_clf = X_train.copy()
    y_train_clf = y_train[target_vars_clf].copy()
    X_test_clf = X_test.copy()
    y_test_clf = y_test[target_vars_clf].copy()
    
    return X_train_reg, y_train_reg, X_test_reg, y_test_reg, X_train_clf, y_train_clf, X_test_clf, y_test_clf

def split_reg_clf(X_train, y_train, target_vars_reg, target_vars_clf):
    """
    Split the data into regression and classification targets.

    Parameters:
    - X_train: Training feature matrix.
    - y_train: Training target vector.
    - target_vars_reg: List of target variables for regression.
    - target_vars_clf: List of target variables for classification.

    Returns:
    - X_train_reg, y_train_reg, X_test_reg, y_test_reg: Regression data.
    - X_train_clf, y_train_clf, X_test_clf, y_test_clf: Classification data.
    """
    # Add target prefix to target variables
    target_vars_reg = [f'{var}_next' for var in target_vars_reg]
    target_vars_clf = [f'{var}_next' for var in target_vars_clf]

    # Regression data
    X_train_reg = X_train.copy()
    y_train_reg = y_train[target_vars_reg].copy()
    
    # Classification data
    X_train_clf = X_train.copy()
    y_train_clf = y_train[target_vars_clf].copy()
    
    return X_train_reg, y_train_reg, X_train_clf, y_train_clf,

In [28]:
from xgboost import XGBClassifier

def train_xgboost_model_reg(X_train, y_train, X_val=None, y_val=None):
    """
    Train an XGBoost model for regression.

    Parameters:
    - X_train: Training feature matrix.
    - y_train: Training target vector.

    Returns:
    - xgb_regressor: Trained XGBoost regressor model.
    """
    xgb_regressor = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=50,
        max_depth=9,
        learning_rate=0.1,
        n_jobs=-1,
        tree_method='hist'  # Faster training
    )

    # if val is none, use 10% of the training data as validation
    if X_val is None:
        val_size = int(0.1 * len(X_train))
        X_val = X_train.iloc[-val_size:]
        y_val = y_train.iloc[-val_size:]

    xgb_regressor.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1)
    return xgb_regressor

def train_xgboost_model_clf(X_train, y_train, X_val=None, y_val=None):
    """
    Train an XGBoost model for classification.

    Parameters:
    - X_train: Training feature matrix.
    - y_train: Training target vector.

    Returns:
    - xgb_classifier: Trained XGBoost classifier model.
    """
    xgb_classifier = XGBClassifier(
        objective='multi:softprob',
        num_class=len(np.unique(y_train)),
        n_estimators=50,
        max_depth=9,
        learning_rate=0.1,
        n_jobs=-1,
        tree_method='hist'
    )

    # if val is none, use 10% of the training data as validation
    if X_val is None:
        val_size = int(0.1 * len(X_train))
        X_val = X_train.iloc[-val_size:]
        y_val = y_train.iloc[-val_size:]

    xgb_classifier.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1)
    return xgb_classifier


In [13]:
def evaluate_model_reg(model, X_test, y_test):
    """
    Evaluate the trained model using RMSE for latitude and longitude.

    Parameters:
    - model: Trained model.
    - X_test: Testing feature matrix.
    - y_test: Testing target vector.
    """

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate RMSE for latitude and longitude
    rmse_lat = np.sqrt(mean_squared_error(y_test['latitude_next'], y_pred[:, 0]))
    rmse_lon = np.sqrt(mean_squared_error(y_test['longitude_next'], y_pred[:, 1]))

    print(f'RMSE Latitude: {rmse_lat:.5f}')
    print(f'RMSE Longitude: {rmse_lon:.5f}')

def evaluate_model_clf(model, X_test, y_test):
    """
    Evaluate the trained model using accuracy.

    Parameters:
    - model: Trained model.
    - X_test: Testing feature matrix.
    - y_test: Testing target vector.
    """

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = np.mean(y_pred == y_test)

    print(f'Accuracy: {accuracy:.5f}')


In [14]:
df = pd.read_csv('../data/xgb_kalman/ais_train_kalman.csv')
df

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,weekday,vessel_embedding,CEU,length,vesselType,latitude_kalman,longitude_kalman,sog_kalman,cog_kalman
0,307.6,17.3,0,7.57302,77.49505,1002660.0,0.0,1583.0,1,14,4,0,6500,199.0,83.0,7.573019,77.495051,17.298020,307.604950
1,306.8,16.9,0,7.65043,77.39404,1004243.0,0.0,1285.0,1,14,4,0,6500,199.0,83.0,7.624642,77.427689,17.033244,307.069205
2,307.9,16.9,0,7.71275,77.31394,1005528.0,0.0,1259.0,1,15,4,0,6500,199.0,83.0,7.679728,77.356573,16.950165,307.588323
3,307.0,16.3,0,7.77191,77.23585,1006787.0,0.0,901.0,1,15,4,0,6500,199.0,83.0,7.736811,77.281816,16.547891,307.224136
4,307.6,16.1,0,7.81285,77.18147,1007688.0,0.0,1211.0,1,15,4,0,6500,199.0,83.0,7.783834,77.219761,16.271176,307.456487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,7.0,1249.0,5,22,1,686,200,191.0,83.0,59.604671,21.507221,13.988585,312.687220
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,7.0,1249.0,5,22,1,686,200,191.0,83.0,59.661060,21.405233,13.563406,319.802466
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,7.0,1219.0,5,23,1,686,200,191.0,83.0,59.724631,21.373044,12.720960,342.482787
1521375,52.6,17.3,0,59.83316,21.38489,11057893.0,7.0,1248.0,5,23,1,686,200,191.0,83.0,59.791703,21.380197,15.551079,163.325389


In [15]:
df = process_navstat(df)
df

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,weekday,vessel_embedding,CEU,length,vesselType,latitude_kalman,longitude_kalman,sog_kalman,cog_kalman
0,307.6,17.3,0,7.57302,77.49505,1002660.0,0.0,1583.0,1,14,4,0,6500,199.0,83.0,7.573019,77.495051,17.298020,307.604950
1,306.8,16.9,0,7.65043,77.39404,1004243.0,0.0,1285.0,1,14,4,0,6500,199.0,83.0,7.624642,77.427689,17.033244,307.069205
2,307.9,16.9,0,7.71275,77.31394,1005528.0,0.0,1259.0,1,15,4,0,6500,199.0,83.0,7.679728,77.356573,16.950165,307.588323
3,307.0,16.3,0,7.77191,77.23585,1006787.0,0.0,901.0,1,15,4,0,6500,199.0,83.0,7.736811,77.281816,16.547891,307.224136
4,307.6,16.1,0,7.81285,77.18147,1007688.0,0.0,1211.0,1,15,4,0,6500,199.0,83.0,7.783834,77.219761,16.271176,307.456487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,7.0,1249.0,5,22,1,686,200,191.0,83.0,59.604671,21.507221,13.988585,312.687220
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,7.0,1249.0,5,22,1,686,200,191.0,83.0,59.661060,21.405233,13.563406,319.802466
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,7.0,1219.0,5,23,1,686,200,191.0,83.0,59.724631,21.373044,12.720960,342.482787
1521375,52.6,17.3,0,59.83316,21.38489,11057893.0,7.0,1248.0,5,23,1,686,200,191.0,83.0,59.791703,21.380197,15.551079,163.325389


In [16]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

n_lag = 20  # Adjust as needed
lagged_vars = [
    'cog', 'sog', 'navstat', 'latitude', 'longitude',
    'time_delta', 'latitude_kalman', 'longitude_kalman', 'cog_kalman', 'sog_kalman'
]
df_lagged = create_lag_features(df, lagged_vars, n_lag)
df_lagged

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,...,sog_kalman_lag11,sog_kalman_lag12,sog_kalman_lag13,sog_kalman_lag14,sog_kalman_lag15,sog_kalman_lag16,sog_kalman_lag17,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20
0,307.6,17.3,0,7.57302,77.49505,1002660.0,0.0,1583.0,1,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,306.8,16.9,0,7.65043,77.39404,1004243.0,0.0,1285.0,1,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,307.9,16.9,0,7.71275,77.31394,1005528.0,0.0,1259.0,1,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,307.0,16.3,0,7.77191,77.23585,1006787.0,0.0,901.0,1,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,307.6,16.1,0,7.81285,77.18147,1007688.0,0.0,1211.0,1,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,7.0,1249.0,5,22,...,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001,0.494001,0.494001
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,7.0,1249.0,5,22,...,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001,0.494001
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,7.0,1219.0,5,23,...,16.183858,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001
1521375,52.6,17.3,0,59.83316,21.38489,11057893.0,7.0,1248.0,5,23,...,16.132731,16.183858,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001


In [17]:
time_vars = ['month', 'hour', 'weekday']
df_lagged = adjust_features_for_prediction_time(df_lagged, time_vars)
df_lagged

,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,...,sog_kalman_lag11,sog_kalman_lag12,sog_kalman_lag13,sog_kalman_lag14,sog_kalman_lag15,sog_kalman_lag16,sog_kalman_lag17,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20
0,307.6,17.3,0,7.57302,77.49505,1002660.0,0.0,1583.0,1.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,306.8,16.9,0,7.65043,77.39404,1004243.0,0.0,1285.0,1.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,307.9,16.9,0,7.71275,77.31394,1005528.0,0.0,1259.0,1.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,307.0,16.3,0,7.77191,77.23585,1006787.0,0.0,901.0,1.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,307.6,16.1,0,7.81285,77.18147,1007688.0,0.0,1211.0,1.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,7.0,1249.0,5.0,22.0,...,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001,0.494001,0.494001
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,7.0,1249.0,5.0,23.0,...,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001,0.494001
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,7.0,1219.0,5.0,23.0,...,16.183858,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001
1521375,52.6,17.3,0,59.83316,21.38489,11057893.0,7.0,1248.0,5.0,23.0,...,16.132731,16.183858,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001


In [18]:
df_lagged = df_lagged.drop(columns=['merged_cluster'], errors='ignore')
df_lagged

,cog,sog,navstat,latitude,longitude,time_numeric,time_delta,month,hour,weekday,...,sog_kalman_lag11,sog_kalman_lag12,sog_kalman_lag13,sog_kalman_lag14,sog_kalman_lag15,sog_kalman_lag16,sog_kalman_lag17,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20
0,307.6,17.3,0,7.57302,77.49505,1002660.0,1583.0,1.0,14.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,306.8,16.9,0,7.65043,77.39404,1004243.0,1285.0,1.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,307.9,16.9,0,7.71275,77.31394,1005528.0,1259.0,1.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,307.0,16.3,0,7.77191,77.23585,1006787.0,901.0,1.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,307.6,16.1,0,7.81285,77.18147,1007688.0,1211.0,1.0,16.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,1249.0,5.0,22.0,1.0,...,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001,0.494001,0.494001
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,1249.0,5.0,23.0,1.0,...,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001,0.494001
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,1219.0,5.0,23.0,1.0,...,16.183858,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001,0.494001
1521375,52.6,17.3,0,59.83316,21.38489,11057893.0,1248.0,5.0,23.0,1.0,...,16.132731,16.183858,15.994022,15.820677,15.690435,15.187374,14.515450,12.720737,6.716862,0.494001


In [220]:
# go through df_lagged if the next vessel_embedding is different from the current one, then we have a new vessel. Store the last row of each vessel in a dict where the key is the vessel_embedding
last_obs_dict = {}
for i in range(len(df_lagged)-1):
    if df_lagged['vessel_embedding'][i] != df_lagged['vessel_embedding'][i+1]:
        last_obs_dict[df_lagged['vessel_embedding'][i]] = df_lagged.iloc[i]

# add the last row of the last vessel to the dict
last_obs_dict[df_lagged['vessel_embedding'].iloc[-1]] = df_lagged.iloc[-1]

# store the dict with the last row of each vessel in a pickle file
with open('../data/xgb_kalman/last_obs.pkl', 'wb') as f:
    pickle.dump(last_obs_dict, f)

In [19]:
target_vars_reg = ['latitude', 'longitude', 'cog', 'sog']
target_vars_clf = ['navstat']
target_vars = target_vars_reg + target_vars_clf
df_lagged = create_target_variables(df_lagged, target_vars)
df_lagged

,cog,sog,navstat,latitude,longitude,time_numeric,time_delta,month,hour,weekday,...,sog_kalman_lag16,sog_kalman_lag17,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20,latitude_next,longitude_next,cog_next,sog_next,navstat_next
0,307.6,17.3,0,7.57302,77.49505,1002660.0,1583.0,1.0,14.0,4.0,...,NaN,NaN,NaN,NaN,NaN,7.65043,77.39404,306.8,16.9,0.0
1,306.8,16.9,0,7.65043,77.39404,1004243.0,1285.0,1.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,7.71275,77.31394,307.9,16.9,0.0
2,307.9,16.9,0,7.71275,77.31394,1005528.0,1259.0,1.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,7.77191,77.23585,307.0,16.3,0.0
3,307.0,16.3,0,7.77191,77.23585,1006787.0,901.0,1.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,7.81285,77.18147,307.6,16.1,0.0
4,307.6,16.1,0,7.81285,77.18147,1007688.0,1211.0,1.0,16.0,4.0,...,NaN,NaN,NaN,NaN,NaN,7.86929,77.11032,309.5,16.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,1249.0,5.0,22.0,1.0,...,6.716862,0.494001,0.494001,0.494001,0.494001,59.69588,21.34225,324.2,13.3,0.0
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,1249.0,5.0,23.0,1.0,...,12.720737,6.716862,0.494001,0.494001,0.494001,59.76388,21.35317,356.5,12.2,0.0
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,1219.0,5.0,23.0,1.0,...,14.515450,12.720737,6.716862,0.494001,0.494001,59.83316,21.38489,52.6,17.3,0.0
1521375,52.6,17.3,0,59.83316,21.38489,11057893.0,1248.0,5.0,23.0,1.0,...,15.187374,14.515450,12.720737,6.716862,0.494001,59.89167,21.54685,53.6,17.7,0.0


In [20]:
lagged_feature_cols = [f'{var}_lag{lag}' for var in lagged_vars for lag in range(1, n_lag + 1)]
required_cols = lagged_feature_cols + [
    'month', 'hour', 'weekday', 'vessel_embedding', 'CEU', 'length', 'vesselType',
    'latitude_next', 'longitude_next', 'cog_next', 'sog_next', 'navstat_next'
]
df_lagged = df_lagged.dropna(subset=required_cols)
df_lagged

,cog,sog,navstat,latitude,longitude,time_numeric,time_delta,month,hour,weekday,...,sog_kalman_lag16,sog_kalman_lag17,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20,latitude_next,longitude_next,cog_next,sog_next,navstat_next
20,297.9,4.7,0,-29.84098,31.15194,2183778.0,1212.0,1.0,6.0,4.0,...,16.271176,16.547891,16.950165,17.033244,17.298020,-29.82024,31.11275,314.0,3.3,0.0
21,314.0,3.3,0,-29.82024,31.11275,2184990.0,1213.0,1.0,7.0,4.0,...,16.165559,16.271176,16.547891,16.950165,17.033244,-29.81090,31.12018,29.6,2.7,0.0
22,29.6,2.7,0,-29.81090,31.12018,2186203.0,1127.0,1.0,7.0,4.0,...,16.063423,16.165559,16.271176,16.547891,16.950165,-29.83246,31.08940,219.6,10.5,0.0
23,219.6,10.5,0,-29.83246,31.08940,2187330.0,1152.0,1.0,7.0,4.0,...,16.024412,16.063423,16.165559,16.271176,16.547891,-29.87551,31.05145,227.9,7.6,0.0
24,227.9,7.6,0,-29.87551,31.05145,2188482.0,106813.0,1.0,13.0,5.0,...,16.071310,16.024412,16.063423,16.165559,16.271176,-29.87645,31.05018,60.1,7.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521371,296.3,14.7,0,59.57721,21.54090,11052917.0,1259.0,5.0,22.0,1.0,...,0.494001,0.494001,0.494001,0.494001,0.494001,59.63337,21.43237,324.1,13.5,0.0
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,1249.0,5.0,22.0,1.0,...,6.716862,0.494001,0.494001,0.494001,0.494001,59.69588,21.34225,324.2,13.3,0.0
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,1249.0,5.0,23.0,1.0,...,12.720737,6.716862,0.494001,0.494001,0.494001,59.76388,21.35317,356.5,12.2,0.0
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,1219.0,5.0,23.0,1.0,...,14.515450,12.720737,6.716862,0.494001,0.494001,59.83316,21.38489,52.6,17.3,0.0


In [21]:
navstat_lag_cols = [f'navstat_lag{lag}' for lag in range(1, n_lag + 1)]
categorical_cols = navstat_lag_cols + ['vesselType']
df_lagged, label_encoders = encode_categorical_variables(df_lagged, categorical_cols)
df_lagged

,cog,sog,navstat,latitude,longitude,time_numeric,time_delta,month,hour,weekday,...,sog_kalman_lag16,sog_kalman_lag17,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20,latitude_next,longitude_next,cog_next,sog_next,navstat_next
20,297.9,4.7,0,-29.84098,31.15194,2183778.0,1212.0,1.0,6.0,4.0,...,16.271176,16.547891,16.950165,17.033244,17.298020,-29.82024,31.11275,314.0,3.3,0.0
21,314.0,3.3,0,-29.82024,31.11275,2184990.0,1213.0,1.0,7.0,4.0,...,16.165559,16.271176,16.547891,16.950165,17.033244,-29.81090,31.12018,29.6,2.7,0.0
22,29.6,2.7,0,-29.81090,31.12018,2186203.0,1127.0,1.0,7.0,4.0,...,16.063423,16.165559,16.271176,16.547891,16.950165,-29.83246,31.08940,219.6,10.5,0.0
23,219.6,10.5,0,-29.83246,31.08940,2187330.0,1152.0,1.0,7.0,4.0,...,16.024412,16.063423,16.165559,16.271176,16.547891,-29.87551,31.05145,227.9,7.6,0.0
24,227.9,7.6,0,-29.87551,31.05145,2188482.0,106813.0,1.0,13.0,5.0,...,16.071310,16.024412,16.063423,16.165559,16.271176,-29.87645,31.05018,60.1,7.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521371,296.3,14.7,0,59.57721,21.54090,11052917.0,1259.0,5.0,22.0,1.0,...,0.494001,0.494001,0.494001,0.494001,0.494001,59.63337,21.43237,324.1,13.5,0.0
1521372,324.1,13.5,0,59.63337,21.43237,11054176.0,1249.0,5.0,22.0,1.0,...,6.716862,0.494001,0.494001,0.494001,0.494001,59.69588,21.34225,324.2,13.3,0.0
1521373,324.2,13.3,0,59.69588,21.34225,11055425.0,1249.0,5.0,23.0,1.0,...,12.720737,6.716862,0.494001,0.494001,0.494001,59.76388,21.35317,356.5,12.2,0.0
1521374,356.5,12.2,0,59.76388,21.35317,11056674.0,1219.0,5.0,23.0,1.0,...,14.515450,12.720737,6.716862,0.494001,0.494001,59.83316,21.38489,52.6,17.3,0.0


In [24]:
X, y, feature_cols, target_cols = prepare_features_and_targets(df_lagged, lagged_vars, n_lag)

In [ ]:
#X_train, X_test, y_train, y_test = split_data(X, y, df_lagged, split_ratio=0.85)

In [96]:
X

,cog_lag1,cog_lag2,cog_lag3,cog_lag4,cog_lag5,cog_lag6,cog_lag7,cog_lag8,cog_lag9,cog_lag10,...,sog_kalman_lag18,sog_kalman_lag19,sog_kalman_lag20,month,hour,weekday,vessel_embedding,CEU,length,vesselType
20,202.2,218.4,265.1,243.4,234.5,222.5,227.6,308.1,297.3,294.2,...,16.950165,17.033244,17.298020,1.0,6.0,4.0,0,6500,199.0,3.0
21,297.9,202.2,218.4,265.1,243.4,234.5,222.5,227.6,308.1,297.3,...,16.547891,16.950165,17.033244,1.0,7.0,4.0,0,6500,199.0,3.0
22,314.0,297.9,202.2,218.4,265.1,243.4,234.5,222.5,227.6,308.1,...,16.271176,16.547891,16.950165,1.0,7.0,4.0,0,6500,199.0,3.0
23,29.6,314.0,297.9,202.2,218.4,265.1,243.4,234.5,222.5,227.6,...,16.165559,16.271176,16.547891,1.0,7.0,4.0,0,6500,199.0,3.0
24,219.6,29.6,314.0,297.9,202.2,218.4,265.1,243.4,234.5,222.5,...,16.063423,16.165559,16.271176,1.0,13.0,5.0,0,6500,199.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521371,292.6,291.3,288.5,288.7,247.4,247.9,247.0,266.5,278.8,278.7,...,0.494001,0.494001,0.494001,5.0,22.0,1.0,686,200,191.0,3.0
1521372,296.3,292.6,291.3,288.5,288.7,247.4,247.9,247.0,266.5,278.8,...,0.494001,0.494001,0.494001,5.0,22.0,1.0,686,200,191.0,3.0
1521373,324.1,296.3,292.6,291.3,288.5,288.7,247.4,247.9,247.0,266.5,...,0.494001,0.494001,0.494001,5.0,23.0,1.0,686,200,191.0,3.0
1521374,324.2,324.1,296.3,292.6,291.3,288.5,288.7,247.4,247.9,247.0,...,6.716862,0.494001,0.494001,5.0,23.0,1.0,686,200,191.0,3.0


In [97]:
y

,latitude_next,longitude_next,sog_next,cog_next,navstat_next
20,-29.82024,31.11275,3.3,314.0,0.0
21,-29.81090,31.12018,2.7,29.6,0.0
22,-29.83246,31.08940,10.5,219.6,0.0
23,-29.87551,31.05145,7.6,227.9,0.0
24,-29.87645,31.05018,7.4,60.1,0.0
...,...,...,...,...,...
1521371,59.63337,21.43237,13.5,324.1,0.0
1521372,59.69588,21.34225,13.3,324.2,0.0
1521373,59.76388,21.35317,12.2,356.5,0.0
1521374,59.83316,21.38489,17.3,52.6,0.0


In [25]:
X_train_reg, y_train_reg, X_train_clf, y_train_clf, = split_reg_clf(X, y, target_vars_reg, target_vars_clf)

In [29]:
xgb_reg = train_xgboost_model_reg(X_train_reg, y_train_reg)

[0]	validation_0-rmse:85.34940
[1]	validation_0-rmse:78.59996
[2]	validation_0-rmse:72.66070
[3]	validation_0-rmse:67.45838
[4]	validation_0-rmse:62.92872
[5]	validation_0-rmse:58.98593
[6]	validation_0-rmse:55.57683
[7]	validation_0-rmse:52.62870
[8]	validation_0-rmse:50.11324
[9]	validation_0-rmse:47.96272
[10]	validation_0-rmse:46.13076
[11]	validation_0-rmse:44.59311
[12]	validation_0-rmse:43.28437
[13]	validation_0-rmse:42.18113
[14]	validation_0-rmse:41.25439
[15]	validation_0-rmse:40.46587
[16]	validation_0-rmse:39.81738
[17]	validation_0-rmse:39.24569
[18]	validation_0-rmse:38.78516
[19]	validation_0-rmse:38.38539
[20]	validation_0-rmse:38.04764
[21]	validation_0-rmse:37.74197
[22]	validation_0-rmse:37.49375
[23]	validation_0-rmse:37.27212
[24]	validation_0-rmse:37.10187
[25]	validation_0-rmse:36.94868
[26]	validation_0-rmse:36.79969
[27]	validation_0-rmse:36.67125
[28]	validation_0-rmse:36.54617
[29]	validation_0-rmse:36.41157
[30]	validation_0-rmse:36.31619
[31]	validation_0-

In [30]:
# Ensure y_train_clf and y_test_clf contain only integer values
y_train_clf = y_train_clf.astype(int)

# Replace unexpected class values with the closest valid class
y_train_clf = y_train_clf.map(lambda x: x if x in [0, 1, 2] else 2)

# Train the classifier
xgb_clf = train_xgboost_model_clf(X_train_clf, y_train_clf)

[0]	validation_0-mlogloss:0.96581
[1]	validation_0-mlogloss:0.85574
[2]	validation_0-mlogloss:0.76278
[3]	validation_0-mlogloss:0.68363
[4]	validation_0-mlogloss:0.61552
[5]	validation_0-mlogloss:0.55662
[6]	validation_0-mlogloss:0.50525
[7]	validation_0-mlogloss:0.46033
[8]	validation_0-mlogloss:0.42084
[9]	validation_0-mlogloss:0.38593
[10]	validation_0-mlogloss:0.35525
[11]	validation_0-mlogloss:0.32792
[12]	validation_0-mlogloss:0.30372
[13]	validation_0-mlogloss:0.28203
[14]	validation_0-mlogloss:0.26286
[15]	validation_0-mlogloss:0.24574
[16]	validation_0-mlogloss:0.23042
[17]	validation_0-mlogloss:0.21687
[18]	validation_0-mlogloss:0.20466
[19]	validation_0-mlogloss:0.19376
[20]	validation_0-mlogloss:0.18388
[21]	validation_0-mlogloss:0.17496
[22]	validation_0-mlogloss:0.16694
[23]	validation_0-mlogloss:0.15976
[24]	validation_0-mlogloss:0.15327
[25]	validation_0-mlogloss:0.14749
[26]	validation_0-mlogloss:0.14225
[27]	validation_0-mlogloss:0.13746
[28]	validation_0-mlogloss:0.1

In [31]:
# save the models
import pickle
with open('../models/xgb_kalman/xgb_reg.pkl', 'wb') as f:
    pickle.dump(xgb_reg, f)

with open('../models/xgb_kalman/xgb_clf.pkl', 'wb') as f:
    pickle.dump(xgb_clf, f)

## Predict on test data

In [65]:
def initialize_lags(last_known_data, lag_vars, n_lags):
    """
    Initialize lagged features using the last known data.
    
    Parameters:
    - last_known_data: DataFrame with the last known data for the vessel.
    - lag_vars: List of variables to create lags for.
    - n_lags: Number of lags.
    
    Returns:
    - lags: Dictionary containing initialized lagged values.
    """
    lags = {}
    for var in lag_vars:
        for lag in range(1, n_lags + 1):
            col_name = f'{var}_lag{lag}'
            if col_name in last_known_data.index:
                lags[col_name] = last_known_data[col_name]
            else:
                raise ValueError(f'Column {col_name} not found in last known data.')
    return lags


In [66]:
def get_static_features(last_known_data):
    """
    Extract static features for the vessel.
    
    Parameters:
    - last_known_data: DataFrame with the last known data for the vessel.
    
    Returns:
    - static_features: Dictionary containing static feature values.
    """
    static_features = {
        'vessel_embedding': last_known_data['vessel_embedding'],
        'CEU': last_known_data['CEU'],
        'length': last_known_data['length'],
        'vesselType': last_known_data['vesselType']
    }
    return static_features


In [58]:
def prepare_features(lags, static_features, month, hour, weekday):
    """
    Combine lags, static features, and time features into a single feature dictionary.
    
    Parameters:
    - lags: Dictionary of lagged features.
    - static_features: Dictionary of static features.
    - month, hour, weekday: Time features.
    
    Returns:
    - features: Dictionary ready for model input.
    """
    features = {}
    features.update(lags)
    features.update(static_features)
    features['month'] = month
    features['hour'] = hour
    features['weekday'] = weekday
    return features

In [82]:
def update_lags(lags, predictions, time_delta, n_lags):
    """
    Update lagged features with the latest predictions and shift previous lags.
    
    Parameters:
    - lags: Dictionary of current lagged features.
    - predictions: Dictionary of current predictions.
    - time_delta: Time delta for 'time_delta_lag1'.
    - n_lags: Number of lags.
    
    Returns:
    - lags: Updated dictionary of lagged features.
    """
    # Shift lags
    for var in predictions.keys():
        for lag in range(n_lags, 1, -1):
            lags[f'{var}_lag{lag}'] = lags[f'{var}_lag{lag-1}']
    
    # Shift 'time_delta' lags
    for lag in range(n_lags, 1, -1):
        lags[f'time_delta_lag{lag}'] = lags[f'time_delta_lag{lag-1}']

    # Update lag1 with current predictions
    for var, value in predictions.items():
        lags[f'{var}_lag1'] = value
    lags['time_delta_lag1'] = time_delta
    return lags


In [67]:
def initialize_kalman_filter(last_known_data, vessel_embedding, kalman_filters_dict):
    """
    Initialize the Kalman filter with the last known state.
    
    Parameters:
    - last_known_data: DataFrame with the last known data for the vessel.
    
    Returns:
    - kf: Initialized Kalman filter instance.
    """
    initial_state = np.array([
        last_known_data['latitude_lag1'],
        last_known_data['longitude_lag1'],
        last_known_data['sog_lag1'],
        last_known_data['cog_lag1']
    ])
    # Create and initialize your Kalman filter here
    kf = kalman_filters_dict[vessel_embedding]
    kf.x = initial_state
    return kf


In [55]:
def update_kalman_filter(kf, measurement, delta_t):
    """
    Update the Kalman filter with the new measurement.
    
    Parameters:
    - kf: Kalman filter instance.
    - measurement: Dictionary containing 'latitude', 'longitude', 'sog', 'cog'.
    - delta_t: Time delta since the last update.
    
    Returns:
    - kf: Updated Kalman filter instance.
    - kalman_estimates: Dictionary with Kalman-filtered estimates.
    """
    # Update the time delta in the filter
    kf.dt = delta_t
    
    # Perform predict and update steps
    kf.predict()
    z = np.array([measurement['latitude'], measurement['longitude'], measurement['sog'], measurement['cog']])
    kf.update(z)
    
    # Extract the Kalman-filtered estimates
    kalman_estimates = {
        'latitude_kalman': kf.x[0],
        'longitude_kalman': kf.x[1],
        'sog_kalman': kf.x[2],
        'cog_kalman': kf.x[3]
    }
    return kf, kalman_estimates


In [90]:
def predict_for_vessel(
    vessel_embedding, vessel_test_times, xgb_reg, xgb_clf, kalman_filters_dict,
    get_last_known_data, expected_feature_columns, last_obs, n_lags=20
):
    """
    Predict latitude and longitude for a single vessel.
    
    Parameters:
    - vessel_id: ID of the vessel.
    - vessel_test_times: DataFrame with 'ID' and 'time' for the vessel.
    - xgb_reg: Trained regression model.
    - xgb_clf: Trained classification model.
    - kalman_filters_dict: Dictionary of Kalman filters for each vessel.
    - get_last_known_data: Function to retrieve last known data for the vessel.
    - expected_feature_columns: List of expected feature column names.
    - last_obs: pd.Series with the last observation for each vessel.
    - n_lags: Number of lags used in the model.
    
    Returns:
    - predictions: List of dictionaries with predictions for each timestamp.
    """
    # Get the last known data for this vessel
    last_known_data = get_last_known_data(last_obs)
    if last_known_data is None or last_known_data.empty:
        print(f"No last known data for vessel {vessel_id}")
        return None
    
    # Initialize lags and static features
    lag_vars = [
        'cog', 'sog', 'latitude', 'longitude', 'navstat',
        'cog_kalman', 'sog_kalman', 'latitude_kalman', 'longitude_kalman'
    ]
    lags = initialize_lags(last_known_data, lag_vars + ['time_delta'], n_lags)
    static_features = get_static_features(last_known_data)

    
    # Initialize Kalman filter
    kf = initialize_kalman_filter(last_known_data, vessel_embedding, kalman_filters_dict)
    
    # Prepare to store predictions
    predictions = []
    previous_time = None
    previous_time_numeric = None
    time_newyear = pd.to_datetime('2024-01-01 00:00:00')
    
    # Iterate over each timestamp
    for idx, row in vessel_test_times.iterrows():
        current_time = pd.to_datetime(row['time']) 
        current_time_numeric = (pd.to_datetime(row['time']) - time_newyear).total_seconds()
        
        # Extract time features
        month = current_time.month
        hour = current_time.hour
        weekday = current_time.weekday()
        
        # Compute time_delta
        if previous_time is not None:
            time_delta = current_time_numeric - previous_time_numeric
        else:
            last_known_time = last_known_data['time_numeric']
            time_delta = current_time_numeric - last_known_time
        
        lags['time_delta_lag1'] = time_delta
        
        # Prepare input features
        features = prepare_features(lags, static_features, month, hour, weekday)
        
        # Convert features to DataFrame and ensure correct column order
        X_input = pd.DataFrame([features])
        X_input = X_input[expected_feature_columns]
        
        # Predict with models
        y_pred_reg = xgb_reg.predict(X_input)
        y_pred_clf = xgb_clf.predict(X_input)
        
        # Extract predictions
        predictions_dict = {
            'latitude': y_pred_reg[0][0],
            'longitude': y_pred_reg[0][1],
            'cog': y_pred_reg[0][2],
            'sog': y_pred_reg[0][3],
            'navstat': y_pred_clf[0]
        }
        
        # Update Kalman filter
        measurement = {
            'latitude': predictions_dict['latitude'],
            'longitude': predictions_dict['longitude'],
            'sog': predictions_dict['sog'],
            'cog': predictions_dict['cog']
        }
        kf, kalman_estimates = update_kalman_filter(kf, measurement, time_delta)
        
        # Update predictions with Kalman estimates
        predictions_dict.update(kalman_estimates)
        
        # Store the prediction
        prediction = {
            'ID': row['ID'],
            'vesselId': vessel_embedding,
            'time': current_time,
            'latitude_pred': predictions_dict['latitude'],
            'longitude_pred': predictions_dict['longitude'],
            'latitude_kalman': predictions_dict['latitude_kalman'],
            'longitude_kalman': predictions_dict['longitude_kalman'],
            # Include other predictions if needed
        }
        predictions.append(prediction)
        
        # Update lags
        lags = update_lags(lags, predictions_dict, time_delta, n_lags)
        
        # Update previous_time
        previous_time = current_time
        previous_time_numeric = current_time_numeric
    
    return predictions


In [38]:
def get_last_known_data(last_obs):

    # the features with names ...lagx must be renamed to ...lagx+1.
    for i in range(20, 0, -1):
        last_obs = last_obs.rename(index = {f'latitude_lag{i}': f'latitude_lag{i+1}',
                                            f'sog_kalman_lag{i}': f'sog_kalman_lag{i+1}',
                                            f'sog_lag{i}': f'sog_lag{i+1}',
                                            f'time_delta_lag{i}':f'time_delta_lag{i+1}',
                                            f'cog_kalman_lag{i}': f'cog_kalman_lag{i+1}',
                                            f'cog_lag{i}': f'cog_lag{i+1}',
                                            f'longitude_kalman_lag{i}': f'longitude_kalman_lag{i+1}',
                                            f'latitude_kalman_lag{i}': f'latitude_kalman_lag{i+1}',
                                            f'longitude_lag{i}': f'longitude_lag{i+1}',
                                            f'navstat_lag{i}': f'navstat_lag{i+1}'})
        
    # delete _lag21
    last_obs = last_obs.drop(['latitude_lag21', 'sog_kalman_lag21', 'sog_lag21', 'time_delta_lag21', 'cog_kalman_lag21', 'cog_lag21', 'longitude_kalman_lag21', 'latitude_kalman_lag21', 'longitude_lag21', 'navstat_lag21'])



    # the features in last_obs 'latitude', 'sog_kalman', 'sog', 'time_delta', 'cog_kalman', 'cog', 'longitude_kalman', 'latitude_kalman', 'longitude', 'navstat' must be renamed to ...lag1
    last_obs = last_obs.rename(index = {'latitude': 'latitude_lag1',
                                                      'sog_kalman': 'sog_kalman_lag1',
                                                      'sog': 'sog_lag1',
                                                      'time_delta': 'time_delta_lag1',
                                                      'cog_kalman': 'cog_kalman_lag1',
                                                      'cog': 'cog_lag1',
                                                      'longitude_kalman': 'longitude_kalman_lag1',
                                                      'latitude_kalman': 'latitude_kalman_lag1',
                                                      'longitude': 'longitude_lag1',
                                                      'navstat': 'navstat_lag1'})

    return last_obs

In [91]:
# Define the expected feature columns
expected_feature_columns = X_train_reg.columns.tolist()

# Get the last known data
test_data = pd.read_csv('../data/ais_test.csv', sep=',')

# Get the vesselId_dict
with open('../data/xgb_kalman/vesselId_dict.pkl', 'rb') as f:
    vesselId_dict = pickle.load(f)

##################### - DEFINITIONS FOR KALMAN FILTER - #####################

from geopy import Point
from geopy.distance import distance

def compute_distance(sog_knots, delta_t_seconds):
    # Convert SOG from knots to meters per second (1 knot = 0.514444 m/s)
    sog_mps = sog_knots * 0.514444
    distance_m = sog_mps * delta_t_seconds
    return distance_m

# Define the hx function
def hx(x):
    return x

# Define the fx function
def fx(x, dt):
    lat = x[0]
    lon = x[1]
    sog = x[2]
    cog = x[3]

    # Ensure SOG and COG are valid
    if sog <= 0 or sog >= 1022 or np.isnan(sog):
        sog = x[2]  # Use previous valid SOG
    if cog < 0 or cog >= 360 or np.isnan(cog):
        cog = x[3]  # Use previous valid COG

    # Compute distance traveled
    distance_m = compute_distance(sog, dt)

    # Use geopy to compute new position
    start_point = Point(lat, lon)
    destination = distance(meters=distance_m).destination(point=start_point, bearing=cog)

    lat_new = destination.latitude
    lon_new = destination.longitude

    # Assume SOG and COG remain the same
    sog_new = sog
    cog_new = cog

    return np.array([lat_new, lon_new, sog_new, cog_new])

with open('../data/xgb_kalman/kalman_filters_dict.pkl', 'rb') as f:
    kalman_filters_dict = pickle.load(f)

##################### - END DEFINITIONS FOR KALMAN FILTER - #####################

# Get the last obs dict
with open('../data/xgb_kalman/last_obs.pkl', 'rb') as f:
    last_obs_dict = pickle.load(f)

# Get the models
xgb_reg = pickle.load(open('../models/xgb_kalman/xgb_reg.pkl', 'rb'))
xgb_clf = pickle.load(open('../models/xgb_kalman/xgb_clf.pkl', 'rb'))

# Run the predictions
all_predictions = []

n_iter = len(test_data['vesselId'].unique())
k = 0

for vessel_id in test_data['vesselId'].unique():
    vessel_test_times = test_data[test_data['vesselId'] == vessel_id].sort_values('time')
    vessel_embedding = vesselId_dict[vessel_id]
    last_obs = last_obs_dict[vessel_embedding]
    vessel_predictions = predict_for_vessel(
        vessel_embedding,
        vessel_test_times,
        xgb_reg,
        xgb_clf,
        kalman_filters_dict,
        get_last_known_data,
        expected_feature_columns,
        last_obs,
        n_lags=20
    )
    if vessel_predictions:
        all_predictions.extend(vessel_predictions)
        
    k += 1
    progress_bar(k, n_iter, prefix='Progress:', suffix='Complete', length=50)

predictions_df = pd.DataFrame(all_predictions)

# Now 'predictions_df' contains the predicted latitude and longitude for each vessel at the specified times.

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [92]:
# save the predictions
predictions_df.to_csv('../data/xgb_kalman/predictions.csv', index = False)

In [42]:
last_obs = last_obs_dict[13]
last_obs = get_last_known_data(last_obs)
print(set(last_obs.index).difference(set(expected_feature_columns)))
print(set(expected_feature_columns).difference(set(last_obs.index)))

{'time_numeric'}
set()


In [94]:

predictions_final = predictions_df[['ID', 'latitude_pred', 'longitude_pred']]

#rename _pred to _predicted
predictions_final = predictions_final.rename(columns = {'latitude_pred': 'latitude_predicted', 'longitude_pred': 'longitude_predicted'})

predictions_final.to_csv('../data/xgb_kalman/predictions_final.csv', index = False)

predictions_final_kalman = predictions_df[['ID', 'latitude_kalman', 'longitude_kalman']]
predictions_final_kalman = predictions_final_kalman.rename(columns = {'latitude_kalman': 'latitude_predicted', 'longitude_kalman': 'longitude_predicted'})

predictions_final_kalman.to_csv('../data/xgb_kalman/predictions_final_kalman.csv', index = False)

In [95]:
# make a new predictions_final that is the average of the predictions_final and predictions_final_kalmanp
predictions_final_avg = pd.DataFrame()
predictions_final_avg['ID'] = predictions_final['ID']
predictions_final_avg['latitude_predicted'] = (predictions_final['latitude_predicted'] + predictions_final_kalman['latitude_predicted']) / 2
predictions_final_avg['longitude_predicted'] = (predictions_final['longitude_predicted'] + predictions_final_kalman['longitude_predicted']) / 2

predictions_final_avg.to_csv('../data/xgb_kalman/predictions_final_avg.csv', index = False)

In [98]:
# merge X and y into one dataframe (on index)

df = pd.merge(X, y, left_index=True, right_index=True)
df

,cog_lag1,cog_lag2,cog_lag3,cog_lag4,cog_lag5,cog_lag6,cog_lag7,cog_lag8,cog_lag9,cog_lag10,...,weekday,vessel_embedding,CEU,length,vesselType,latitude_next,longitude_next,sog_next,cog_next,navstat_next
20,202.2,218.4,265.1,243.4,234.5,222.5,227.6,308.1,297.3,294.2,...,4.0,0,6500,199.0,3.0,-29.82024,31.11275,3.3,314.0,0.0
21,297.9,202.2,218.4,265.1,243.4,234.5,222.5,227.6,308.1,297.3,...,4.0,0,6500,199.0,3.0,-29.81090,31.12018,2.7,29.6,0.0
22,314.0,297.9,202.2,218.4,265.1,243.4,234.5,222.5,227.6,308.1,...,4.0,0,6500,199.0,3.0,-29.83246,31.08940,10.5,219.6,0.0
23,29.6,314.0,297.9,202.2,218.4,265.1,243.4,234.5,222.5,227.6,...,4.0,0,6500,199.0,3.0,-29.87551,31.05145,7.6,227.9,0.0
24,219.6,29.6,314.0,297.9,202.2,218.4,265.1,243.4,234.5,222.5,...,5.0,0,6500,199.0,3.0,-29.87645,31.05018,7.4,60.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521371,292.6,291.3,288.5,288.7,247.4,247.9,247.0,266.5,278.8,278.7,...,1.0,686,200,191.0,3.0,59.63337,21.43237,13.5,324.1,0.0
1521372,296.3,292.6,291.3,288.5,288.7,247.4,247.9,247.0,266.5,278.8,...,1.0,686,200,191.0,3.0,59.69588,21.34225,13.3,324.2,0.0
1521373,324.1,296.3,292.6,291.3,288.5,288.7,247.4,247.9,247.0,266.5,...,1.0,686,200,191.0,3.0,59.76388,21.35317,12.2,356.5,0.0
1521374,324.2,324.1,296.3,292.6,291.3,288.5,288.7,247.4,247.9,247.0,...,1.0,686,200,191.0,3.0,59.83316,21.38489,17.3,52.6,0.0


In [101]:
# save df
df.to_csv('../data/xgb_kalman/X_y.csv', index = False)

In [100]:
from autogluon.tabular import TabularPredictor

# Define the problem type
problem_type = 'regression'

# Define the label columns
labels = ['latitude_next', 'longitude_next', 'sog_next', 'cog_next', 'navstat_next']

# Define the hyperparameters
hyperparameters = {
    'GBM': {},
    'CAT': {},
    'RF': {},
    'XT': {},
    'KNN': {},
    'NN': {},
    'FASTAI': {},
    'XGB': {}
}

# Create a TabularPredictor
predictor = TabularPredictor(
    label=labels,
    problem_type=problem_type,
    eval_metric='root_mean_squared_error',
    path='autogluon_ais'
)

# Fit the predictor
predictor.fit(df)

# Save the predictor
predictor.save('autogluon_ais')

ModuleNotFoundError: No module named 'autogluon'